## COMP6713  - Natural Language Processing – 25T1

### Tutorial – Week 10


You will learn to combine two approaches for summarization discussed in the class: namely longformer and textrank. Work through cells and comments marked as “#TODO” in the notebook.

The completion of this tutorial will enable you with skills in augmenting statistical and neural models to create hybrid summarizers. You will also learn to encapsulate your code as a class, so that repeated model loading is avoided.

You may use HuggingFace documentation and the demo notebooks provided in class.

In [20]:
from summa.summarizer import summarize
import math

In [21]:
import bs4
import sys
import requests

res = requests.get('https://en.wikipedia.org/wiki/Agriculture' )
res.raise_for_status()
wiki = bs4.BeautifulSoup(res.text,"html.parser")
document = ""
for i in wiki.select('p'):
    document += i.getText().strip()+" "

In [22]:
document

' Agriculture encompasses crop and livestock production, aquaculture, fisheries, and forestry for food and non-food products.[1] Agriculture was the key development in the rise of sedentary human civilization, whereby farming of domesticated species created food surpluses that enabled people to live in cities. While humans started gathering grains at least 105,000 years ago, nascent farmers only began planting them around 11,500 years ago. Sheep, goats, pigs, and cattle were domesticated around 10,000 years ago. Plants were independently cultivated in at least 11 regions of the world. In the 20th century, industrial agriculture based on large-scale monocultures came to dominate agricultural output. As of 2021[update], small farms produce about one-third of the world\'s food, but large farms are prevalent.[2] The largest 1% of farms in the world are greater than 50 hectares (120 acres) and operate more than 70% of the world\'s farmland.[2] Nearly 40% of agricultural land is found on far

In [23]:
text = document

In [30]:
extract = summarize(text, ratio=0.05)

In [31]:
len(text)

64908

In [32]:
len(extract)

16145

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
from transformers import LongformerTokenizer, EncoderDecoderModel

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16").to(device)
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")



config.json:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.bias', 'encoder.embeddings.position_ids', 'decoder.roberta.pooler.dense.weight']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:

input_ids = tokenizer(extract, return_tensors="pt").input_ids
output_ids = model.generate(input_ids.to(device))


/Users/z3539958/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [34]:

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

The agricultural output of China is the largest in the world, followed by the European Union, India and the United States.
In the U.S. of America, more than half of all hired farm workers were immigrants in 2009.
The food demand for agriculture is increasing, and agriculture is facing growing freshwater demand.
Farmers fill labour shortages in high-value agriculture activities, such as farming.
